In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print(tf.__version__)

In [ ]:
cols = ['price', 'maint', 'doors', 'persons', 'lug_capacity', 'safety','output']
cars = pd.read_csv('/data/car_evaluation.csv',names=cols, header=None)

In [ ]:
cars.head()

In [ ]:
plot_size = plt.rcParams["figure.figsize"]
plot_size [0] = 8
plot_size [1] = 6
plt.rcParams["figure.figsize"] = plot_size

In [ ]:
cars.output.value_counts().plot(kind='pie', autopct='%0.05f%%', colors=['lightblue', 'lightgreen', 'orange', 'pink'], explode=(0.05, 0.05, 0.05,0.05))


In [ ]:
price = pd.get_dummies(cars.price, prefix='price')
maint = pd.get_dummies(cars.maint, prefix='maint')

doors = pd.get_dummies(cars.doors, prefix='doors')
persons = pd.get_dummies(cars.persons, prefix='persons')

lug_capacity = pd.get_dummies(cars.lug_capacity, prefix='lug_capacity')
safety = pd.get_dummies(cars.safety, prefix='safety')

labels = pd.get_dummies(cars.output, prefix='condition')

In [ ]:
X = pd.concat([price, maint, doors, persons, lug_capacity, safety] , axis=1)

In [ ]:
labels.head()

In [ ]:
y = labels.values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model

In [ ]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(15, activation='relu')(input_layer)
dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)
output = Dense(y.shape[1], activation='softmax')(dense_layer_2)

model_tf = Model(inputs=input_layer, outputs=output)
model_tf.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
print(model_tf.summary())

In [ ]:
history = model_tf.fit(X_train, y_train, batch_size=8, epochs=30, verbose=1, validation_split=0.2)

In [ ]:
score = model_tf.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
#Model function
def model(algorithm,dtrain_X,dtrain_Y,dtest_X,dtest_Y,cols=None):

    algorithm.fit(dtrain_X,dtrain_Y)
    predictions = algorithm.predict(dtest_X)
    prediction_probabilities = algorithm.predict(dtest_X)
    print (algorithm)
    
    return predictions,prediction_probabilities,algorithm

In [2]:
y_pred,y_prob,model_obj=model(model_tf,X_train,y_train,X_test,y_test,X_train.columns)

NameError: name 'model' is not defined

In [3]:
from mosaicml import *
from mosaicml.constants import MLModelFlavours

/opt/conda/lib/python3.7/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [4]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    print (prediction)
    return prediction.tolist()

In [5]:
import requests
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json(orient = 'records')}

NameError: name 'X_train' is not defined

In [ ]:
req.json['payload']

In [ ]:
labels.head()

In [ ]:
sch = generate_schema(score,(model_obj, req),X_train)
print(sch)

In [ ]:
register_model(model_obj, score, name="CarEvaluation_ClassificationTENSORFLOW", 
                description="Classify Car basis the feature",
                flavour=MLModelFlavours.tensorflow, schema=sch,
                y_true=y_test, y_pred = y_pred, #y_pred=pd.Series(y_pred), 
                prob=y_pred, features=X_train.columns, 
               labels=["condition_acc","condition_good","condition_unacc","condition_vgood"], 
               init_script="" ,
                model_type="classification", 
                input_type="json", explain_ai=True, x_train=X_train, x_test=X_test, 
                y_train=y_train, y_test=y_test,
                feature_names=list(X_train.columns),feature_ids=list(X_train.columns),
                target_names=["condition_acc","condition_good","condition_unacc","condition_vgood"], 
                kyd=True, kyd_score =True)
print("Registering model")